In [ ]:
## add ignore warnings for now, will remove and debug once full algorithm is complete
import warnings
warnings.filterwarnings("ignore")

## import packages/libraries
from time import perf_counter, process_time, clock_gettime_ns, CLOCK_REALTIME
import numpy as np
import pandas as pd
from itertools import product
from multiprocessing import Pool, cpu_count
import sys
import sqlite3
# import cv2 as cv

from cProfile import Profile
from pstats import SortKey, Stats

## append filepath to allow files to be called from within project folder
sys.path.append('/home/gerard/Desktop/capstone_project/patoms')
sys.path.append('/home/gerard/Desktop/capstone_project')

## call locally created functions
from snapshot_2d_pattern_v7 import patoms2d
from pattern_2d_compare_v7 import pattern_compare_2d
from updating_ref_table_v0 import update_ref_table

In [9]:
## access the camera to get video stream
# cap = cv.VideoCapture(0)

In [10]:
# connect to 2d database
con2d = sqlite3.connect("ref2d_v5.db")
cur2d = con2d.cursor()

In [11]:
# read 2d ref database into memory and convert to ref patoms for comparison
working_ref_patoms = []
con2dref = sqlite3.connect("ref2d_v5.db")
cur2dref = con2dref.cursor()
ref_names = [name for (name,) in cur2dref.execute("select name from sqlite_master where type='table' and name like '%ref%';").fetchall()][:5]
for i in ref_names:
    table = cur2dref.execute(f"select * from {i};").fetchall()
    table_array = np.array(table).astype(np.float32)
    working_ref_patoms.append(table_array)

ref_patoms_array = np.vstack(working_ref_patoms).astype('float32')
ref_indices = np.unique(ref_patoms_array[:,7],axis=0)


In [12]:
# print(ref_patoms_array.dtype)

In [ ]:
with Profile() as profile:
    s = perf_counter(), process_time()
    val = 0
    while val < 3:
        frame = np.random.randint(0, 256, (720, 1280), dtype=np.uint8) # 307200 bytes
        # print(frame.nbytes)
        # ret, frame = cap.read()
        # frame = (frame[..., 0] << 16) | (frame[..., 1] << 8) | frame[..., 2]
        # print(frame.nbytes)
        frame = (frame - frame.min()) / (frame.max() - frame.min())
        x_len = frame.shape[0]
        y_len = frame.shape[1]
        ####################### FIRST TASK: FIND PATTERNS IN FRAME ######################
        frame_patoms = patoms2d(x_len, y_len, frame, val)
        # output: [colour, norm_x, norm_y, pattern_centroid_x, pattern_centroid_y, quad, quad_cnt, frame_ind_arr]
        #frame_patoms_arr = np.vstack((frame_patoms))
        num_patoms = len(frame_patoms)
        ############## SECOND TASK: COMPARE NEW PATOMS AGAINST REF PATOMS ###############
        atime = perf_counter(), process_time()
        with Pool(processes=8) as pool:
            items = [(frame_patoms[i], ref_patoms_array) for i in range(num_patoms)]
            comp_results = pool.starmap(pattern_compare_2d, items) #e.g. ['pcol','px','py','pxc','pyc','pq','pqlen','pfind','xc_d','yc_d','x_d','y_d','rfind','similar']
            print("Real Time to compare 2D patterns with multiprocessing (mins):", (perf_counter()-atime[0])/60)
            print("CPU Time to compare 2D patterns with multiprocessing (mins):", (perf_counter()-atime[1])/60)
            # loop through the output of the comparison function
            witime = perf_counter(), process_time()
            for ix, i in enumerate(comp_results):
                table_data = np.unique(i[:,:-2], axis=0)
                existing_tables = [names for (names,) in cur2d.execute("select name from sqlite_master where type='table' and name not like '%ref%';").fetchall()]
                next_table_num = int(existing_tables.pop(-1)[-6:]) + 1 
                next_table_num = str(next_table_num).zfill(6)
                ref_patom_ind = i[:,12].astype('int32')
                ref_patom_ind = np.unique(ref_patom_ind).tolist() # contains list of all ref patoms used in comparison
                #extract reference tables that are similar to the patom in the current iteration
                match_patoms = i[i[:,13] == 1]
                if match_patoms.shape[0] > 0:
                    for k in ref_patom_ind:
                        table_num = ref_names[k][-6:]
                        cur2d.executemany(f"INSERT INTO pat_2d_{table_num}(colour, x_pos_dist, y_pos_dist, x_cent, y_cent, quad, quad_cnt, frame_ind, \
                                            xc_d, yc_d, x_d, y_d) VALUES (?,?,?,?,?,?,?,?,?,?,?,?)", table_data)
                # if no matching ref patoms, write new patom to new table
                else:
                    cur2d.execute(f"CREATE TABLE pat_2d_{next_table_num}(colour, x_pos_dist, y_pos_dist, x_cent, y_cent, quad, quad_cnt, frame_ind, \
                                xc_d, yc_d, x_d, y_d);")
                    cur2d.executemany(f"INSERT INTO pat_2d_{next_table_num}(colour, x_pos_dist, y_pos_dist, x_cent, y_cent, quad, quad_cnt, frame_ind, \
                                    xc_d, yc_d, x_d, y_d) VALUES (?,?,?,?,?,?,?,?,?,?,?,?)", table_data)    
                
                con2d.commit()
            ## major bottleneck is the insert/create steps
            print("Real Time to write/insert patoms to existing/new tables in database (secs):", (perf_counter()-witime[0]))
            print("CPU Time to write/insert patoms to existing/new tables in database (secs):", (perf_counter()-witime[1]))
        val += 1

    print("Real Time to process 1 second of data against 5 reference patoms(mins):", (perf_counter()-s[0])/60)
    print("CPU Time to process 1 second of data against 5 reference patoms(mins):", (perf_counter()-s[1])/60)
    #print("Time to get patoms from 1 seconds of data (mins):", (perf_counter()-s)/60)
    con2d.close()
    con2dref.close()

    (
        Stats(profile)
        .strip_dirs()
        .sort_stats(SortKey.CALLS)
        .print_stats()
    )

Time to get 2D patterns with multiprocessing (secs): 0.6244216289996984
Time to compare 2D patterns with multiprocessing (mins): 0.13012252299995453
Time to write/insert patoms to existing/new tables in database (secs): 119.34343546799937
Time to get 2D patterns with multiprocessing (secs): 0.6642467620040406
Time to compare 2D patterns with multiprocessing (mins): 0.12979810988338916
Time to write/insert patoms to existing/new tables in database (secs): 119.1863969709957
Time to get 2D patterns with multiprocessing (secs): 0.6238237419966026
Time to compare 2D patterns with multiprocessing (mins): 0.12763369926663776
Time to write/insert patoms to existing/new tables in database (secs): 119.06205859200418
Time to get 2D patterns with multiprocessing (secs): 0.5292813599953661
Time to compare 2D patterns with multiprocessing (mins): 0.12990588763338262
Time to write/insert patoms to existing/new tables in database (secs): 119.64832868699887
Time to get 2D patterns with multiprocessing 

In [14]:
########################################################################################################
############## next steps is reconcile ref patoms and put time order on patom sequence #################
########################################################################################################

# fileds in tables: [x_pos_dist, y_pos_dist, x_cent, y_cent, quad, quad_cnt, patom_ind, frame_ind, patom_time, xc_d, yc_d, x_d, y_d]
# working_tables = []
# with open('/home/gerard/Desktop/capstone_project/ref2d_v4.db', 'rb') as f:
#     con2d = sqlite3.connect(":memory:")
#     con2d.deserialize(f.read())
#     cur2d = con2d.cursor()
#     nonref_names = [name for (name,) in cur2d.execute("select name from sqlite_master where type='table' nd name not like '%ref%';").fetchall()]
#     for i in nonref_names:
#         row = cur2d.execute(f"select * from {i} LIMIT 1;").fetchone()
#         #table_array = np.array(table)
#         if row is None:
#             pass
#         else:
#             table = cur2d.execute(f"select * from {i};").fetchall()
#             working_tables.append(table)
            # within table 

## brainstorm idea is the have different databases for each sequential element
## then within each database have different patoms that occur most frequently at that time point
## what about patoms that don't occur in the same time order, or occur at different stages?
## how can we add sequence numbers to patoms so that over time they result in the most frequenct sequence order number
## what scale am I using? create small sequence of say 1 second (30 fps), should that sequnce then be in a single database?
## LOOKS LIKE I'VE COMPLETELY IGNORED 3D PATOMS?????
## we link snapsot sequences via their timestamp
## a number of ref patom tables will link up to 
##  maybe focus on getting streaming video to run through the above model, rather than fake data?